In [1]:
import pandas as pd
import scipy.stats as stats

In [2]:
df1 = pd.read_csv(r'C:\Users\UserK\Desktop\Ranee\data\ML\14_Data.csv')

In [3]:
print(df1.shape)
df1.info()

(26283, 25)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26283 entries, 0 to 26282
Data columns (total 25 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   방송Code    26283 non-null  float64
 1   채널        26283 non-null  object 
 2   소요분       26283 non-null  float64
 3   가중분       26283 non-null  float64
 4   방송구분      26283 non-null  object 
 5   프로그램명     26283 non-null  object 
 6   상품ID      26283 non-null  int64  
 7   상품명       26283 non-null  object 
 8   매입과세구분    26283 non-null  object 
 9   상품목표취급금액  26283 non-null  int64  
 10  상품목표주문금액  26283 non-null  int64  
 11  판매단가      26283 non-null  int64  
 12  수수료율      26283 non-null  float64
 13  상품주문수량    26283 non-null  int64  
 14  상품주문금액    26283 non-null  int64  
 15  상품취소수량    26283 non-null  int64  
 16  상품취소금액    26283 non-null  int64  
 17  ARS금액     26283 non-null  int64  
 18  매입형태      26283 non-null  object 
 19  배송방식      26283 non-null  object 
 20  상품소요분     26283 

1. '방송구분'이라는 항목에서 '녹화방송' 에 해당 하는 데이터의 '프로그램명'별 '상품목표주문금액'과 '상품주문금액'의 합과 평균을 계산하시오.

In [4]:
df1['방송구분'].value_counts()

방송구분
생방송     18706
재방송      7538
녹화방송       39
Name: count, dtype: int64

In [5]:
cond1 = (df1['방송구분'] == '녹화방송')
pv1 = df1.loc[cond1].pivot_table(index='프로그램명', values=['상품목표주문금액','상품주문금액'], aggfunc=['sum','mean'])
# 왜......... mean값 이난리

In [6]:
pv1 = df1.loc[cond1].pivot_table(index='프로그램명', values=['상품목표주문금액','상품주문금액'], aggfunc=['sum','mean'])
df1['상품목표주문금액'].describe()

count    2.628300e+04
mean     7.169959e+07
std      8.129376e+07
min      0.000000e+00
25%      1.575876e+07
50%      4.971633e+07
75%      1.033874e+08
max      1.229786e+09
Name: 상품목표주문금액, dtype: float64

2. '프로그램명'에서 가장 '상품주문금액'의 합이 큰 3가지를 확인하여, 3가지 항목에 대해 데이터를 추출한 뒤, '상품명'별 '상품주문금액'의 합을 계산하시오.

In [7]:
pv1.columns

MultiIndex([( 'sum', '상품목표주문금액'),
            ( 'sum',   '상품주문금액'),
            ('mean', '상품목표주문금액'),
            ('mean',   '상품주문금액')],
           )

In [8]:
top3 = pv1.sort_values(by=( 'sum',   '상품주문금액'),ascending=False).index[0:3]

In [9]:
df2 = df1.loc[df1['프로그램명'].isin(top3)]
df2['프로그램명'].value_counts()

프로그램명
재방_의류      831
재방_신선농산    652
재방_이미용     468
Name: count, dtype: int64

In [10]:
pv2 = df2.pivot_table(index='상품명', values='상품주문금액', aggfunc='sum')
pv2

,상품주문금액
상품명,
(1)[나인테일즈] 링클 파워 세럼 1+1 패키지 (고객요청패키지),164314981
(1)[나인테일즈] 링클 파워 세럼 1세트 패키지 (무료체험포함),180981265
(18년수확)철원오대쌀 10kg + 10kg,70977036
(2018년 수확)김연도 웰빙 혼합곡 + 옹기 미니쌀독,99443164
(D) 클린샤워 프리미엄 때혁명 각질제거제 풀세트,309793170
...,...
향기의 미술관 향수컬렉션,77983457
황금대추 방울토마토3kg,60905159
황토방 청도 감말랭이 세트(30봉),50232365


3. '방송시작시간'과 '방송종료시간'의 차이를 계산하여, '방송진행시간'이라는 항목을 선언하고, '방송진행시간'이 1시간 미만인 데이터의 개수를 확인하시오.

In [11]:
df1['방송진행시간'] = pd.to_datetime(df1['방송종료시간']) - pd.to_datetime(df1['방송시작시간'])

In [12]:
count = 0
for i in df1['방송진행시간'] :
 if i < pd.Timedelta(hours=1) :
    count += 1
count

5769

In [13]:
cond1 = df1['방송진행시간'] < pd.to_timedelta(1, unit='h')
df1.loc[cond1].shape[0]

5769

4. '방송시작시간'을 날짜형식으로 변환하여, '방송연도'/'방송월'/'방송일'/'방송요일'항목을 생성

In [96]:
df1['방송연도'] = pd.to_datetime(df1['방송시작시간']).dt.year
df1['방송월'] = pd.to_datetime(df1['방송시작시간']).dt.month
df1['방송일'] = pd.to_datetime(df1['방송시작시간']).dt.day
df1['방송요일'] = pd.to_datetime(df1['방송시작시간']).dt.day_name()

5. '방송요일'항목에서 주말과 주중을 구분하여, '주말'과 '주중'의 '상품주문수량'과 '상품주문금액'의 평균의 차이를 계산

In [98]:
df1['방송요일'].unique()

array(['Friday', 'Saturday', 'Sunday', 'Monday', 'Tuesday', 'Wednesday',
       'Thursday'], dtype=object)

In [101]:
cond1 = (df1['방송요일'].isin(['Saturday', 'Sunday']))
df_week = df1.loc[~cond1] # 주중
df_end = df1.loc[cond1] # 주말

# 주중과 주말의 상품주문수량과 상품주문금액의 정규성확인
# 귀무가설 : 정규성을 따른다.
# 대립가설 : 정규성을 따르지 않는다.
print(stats.normaltest(df_week['상품주문수량']))
print(stats.normaltest(df_week['상품주문금액']))
print(stats.normaltest(df_end['상품주문수량']))
print(stats.normaltest(df_end['상품주문금액']))

# < 0.05 -> 귀무가설 기각 -> 정규성을 따르지 않는다 -> 중앙값비교 -> Ranksum

NormaltestResult(statistic=6183.227591499221, pvalue=0.0)
NormaltestResult(statistic=19047.731539144883, pvalue=0.0)
NormaltestResult(statistic=2980.303190641076, pvalue=0.0)
NormaltestResult(statistic=7659.412973449251, pvalue=0.0)


In [102]:
# 귀무가설 : 중앙값은 차이가 없다
# 대립가설 : 중앙값은 차이가 있다

print(stats.ranksums(df_week['상품주문수량'],df_end['상품주문수량']))
print(stats.ranksums(df_week['상품주문금액'],df_end['상품주문금액']))

# < 0.05 -> 귀무가설 기각 -> 중앙값 차이가 있다.

RanksumsResult(statistic=-3.7987233869844674, pvalue=0.0001454433050965593)
RanksumsResult(statistic=4.596803679391717, pvalue=4.290215043465792e-06)


6. '배송방식'과 '매입형태'간의 연관성이 있는지 확인하고자 한다.

In [105]:
# '배송방식'과 '매입형태' 연관성이 없다
# '배송방식'과 '매입형태' 연관성이 있다

c1 = pd.crosstab(df1['배송방식'],df1['매입형태'])
stats.chi2_contingency(c1)

# < 0.05 -> 귀무가설 기각 -> 연관성이 있다

Chi2ContingencyResult(statistic=26256.193038804344, pvalue=0.0, dof=2, expected_freq=array([[  980.45428604,    39.54571396],
       [  957.38477343,    38.61522657],
       [23326.16094053,   940.83905947]]))

7. '방송구분'에서 '생방송'과 '녹화방송'의 매출을 비교하고자 한다. '생방송'과'녹화방송'의 '상품주문금액'의 평균의 차이가 있는지 검정

In [107]:
# 상품주문금액이라는 연속형 변수가 정규성을 따르는지 비교
cond1 = (df1['방송구분'] == '생방송')
cond2 = (df1['방송구분'] == '녹화방송')

# 귀무가설 : 정규성 따른다
# 대립가설 : 정규성 따르지 않는다

print(stats.normaltest(df1.loc[cond1]['상품주문금액']))
print(stats.normaltest(df1.loc[cond2]['상품주문금액']))

# < 0.05 -> 귀무가설 기각 -> 정규성을 따르지 않는다 -> 중앙값비교 -> Ranksum

NormaltestResult(statistic=17929.951096292796, pvalue=0.0)
NormaltestResult(statistic=37.59192300234786, pvalue=6.870963444477694e-09)


In [108]:
# 귀무가설 : 중앙값 차이 없다
# 대립가설 : 중앙값 차이 있다

stats.ranksums(df1.loc[cond1]['상품주문금액'],df1.loc[cond2]['상품주문금액'])
# < 0.05 -> 귀무가설 기각 -> 중앙값 차이 있다

RanksumsResult(statistic=5.897121686301845, pvalue=3.698970792035826e-09)

8. '상품목표주문금액'에서 '상품주문금액'의 차이를 계산해, 목표를 달성하지 못한 방송에 대해서는 0 / 목표를 달성한 방송은 1값으로 '목표달성여부' 변수를 생성

In [18]:
df1['목표금액과차이'] = df1['상품목표주문금액'] - (df1['상품주문수량'] * df1['상품주문금액'])
def solution(row) :
    if row <= 0 :
        return 1
    else : return 0
df1['목표달성여부'] = df1['목표금액과차이'].apply(solution)

df1['목표달성여부'].value_counts()

목표달성여부
1    24652
0     1631
Name: count, dtype: int64

In [17]:
df1['목표달성액'] =   df1['상품목표주문금액'] - df1['상품주문금액']
cond1 = df1['목표달성액'] > 0
df1.loc[cond1, '목표달성여부2'] = 1
df1.loc[~cond1, '목표달성여부2'] = 0
df1['목표달성여부2'].value_counts()

목표달성여부2
1.0    16333
0.0     9950
Name: count, dtype: int64

In [124]:
df1['목표달성여부'].value_counts()

목표달성여부
1    24652
0     1631
Name: count, dtype: int64

9. '목표달성여부'를 분류하는 모델을 만들고자 한다.
    - X : 소요분 / 방송구분 / 판매단가 / ARS 금액 /수수료율 / '방송요일' / '방송월'
    - Y : '목표달성여부'
    - 학습 : 검증 = 8 : 2
    - 특성 공학 기법 (결측처리(평균,최빈값) + 스케일링&인코딩)
    - 알고리즘 ( Logistic Regression 회귀 적용 / 하이퍼파라미터 튜닝 적용)
    - 평가

In [152]:
from sklearn.model_selection import train_test_split #
from sklearn.pipeline import make_pipeline # 파이프라인 구성
from sklearn.compose import make_column_transformer # 분할처리 때 필요(병렬로 배치)

from sklearn.impute import SimpleImputer # 결측값 처리
from sklearn.preprocessing import StandardScaler # 스케일링
from sklearn.preprocessing import OneHotEncoder # 인코딩

from imblearn.combine import SMOTEENN # 불균형데이터처리

from sklearn.linear_model import LogisticRegression 
from sklearn.model_selection import GridSearchCV # 교차검증&매개변수 튜닝
from sklearn.metrics import classification_report # 평가

In [153]:
df1.columns

Index(['방송Code', '채널', '소요분', '가중분', '방송구분', '프로그램명', '상품ID', '상품명', '매입과세구분',
       '상품목표취급금액', '상품목표주문금액', '판매단가', '수수료율', '상품주문수량', '상품주문금액', '상품취소수량',
       '상품취소금액', 'ARS금액', '매입형태', '배송방식', '상품소요분', '상품가중분', '상품방송순번', '방송시작시간',
       '방송종료시간', '방송진행시간', '방송연도', '방송월', '방송일', '방송요일', '목표금액과차이', '목표달성여부'],
      dtype='object')

In [154]:
X = df1[['소요분','방송구분','판매단가','ARS금액','수수료율','방송요일','방송월']]
Y = df1['목표달성여부']

In [155]:
Y.isnull().sum()

0

In [156]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26283 entries, 0 to 26282
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   소요분     26283 non-null  float64
 1   방송구분    26283 non-null  object 
 2   판매단가    26283 non-null  int64  
 3   ARS금액   26283 non-null  int64  
 4   수수료율    26283 non-null  float64
 5   방송요일    26283 non-null  object 
 6   방송월     26283 non-null  int32  
dtypes: float64(2), int32(1), int64(2), object(2)
memory usage: 1.3+ MB


In [157]:
X_test, X_train, Y_test, Y_train = train_test_split(X,Y,random_state=1234,test_size=0.8)

In [158]:
numeric_list = X.describe().columns
category_list = X.describe(include='object').columns

In [162]:
numeric_pipe = make_pipeline(SimpleImputer(strategy='mean'), StandardScaler())
category_pipe = make_pipeline(SimpleImputer(strategy='most_frequent'), OneHotEncoder())
prepro_pipe = make_column_transformer((numeric_pipe,numeric_list),
                            (category_pipe,category_list))

In [163]:
prepro_pipe

ColumnTransformer(transformers=[('pipeline-1',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer()),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 Index(['소요분', '판매단가', 'ARS금액', '수수료율', '방송월'], dtype='object')),
                                ('pipeline-2',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehotencoder',
                                                  OneHotEncoder())]),
                                 Index(['방송구분', '방송요일'], dtype='object'))])

In [164]:
model_pipe = make_pipeline(prepro_pipe, LogisticRegression())
model_pipe

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('pipeline-1',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer()),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  Index(['소요분', '판매단가', 'ARS금액', '수수료율', '방송월'], dtype='object')),
                                                 ('pipeline-2',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder())]),
                                                  Index(['방송구분', '방송요일'], dtype='object'))])),
                ('logisticregression', LogisticRegression())])

In [165]:
grid_model = GridSearchCV(model_pipe, param_grid={}, cv=5)
grid_model.fit(X_train,Y_train)
best_model = grid_model.best_estimator_

In [166]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error,mean_absolute_error

In [167]:
Y_train_pred = best_model.predict(X_train)
Y_test_pred = best_model.predict(X_test)

In [168]:
print('학습 결정 계수 : ' , r2_score(Y_train, Y_train_pred))
print('일반화 결정 계수 : ' , r2_score(Y_test, Y_test_pred))

학습 결정 계수 :  0.25776235532975833
일반화 결정 계수 :  0.17652439024390254


In [169]:
print('학습 MSE : ' , mean_squared_error(Y_train, Y_train_pred))
print('일반화 MSE : ' ,  mean_squared_error(Y_test, Y_test_pred))

학습 MSE :  0.042897227374328245
일반화 MSE :  0.049277016742770165


In [170]:
print('학습 MAE : ' , mean_absolute_error(Y_train, Y_train_pred))
print('일반화 MAE : ' ,  mean_absolute_error(Y_test, Y_test_pred))

학습 MAE :  0.042897227374328245
일반화 MAE :  0.049277016742770165
